In [1]:
import torch
import glob
from PIL import Image
import torch.optim as optim
from torch.utils.data import DataLoader,  TensorDataset, Dataset
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt
import math

#import os
import shutil
import time
from tqdm import tqdm

#from torchvision import models, datasets, transforms

In [2]:
class MyDataset(Dataset):
    
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.data.sort()
        self.targets.sort()
        self.transform = transform
        
    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.targets[idx]
        if self.transform:
            image = self.transform(image)
        # sample = {"image": image, "label": label}

        return image, label

In [3]:
# load data

data = np.load('train_val_cropped dataset_1003.npz')
x_images = torch.from_numpy(data['arr_0']/255)
#x_images = torch.transpose(x_images, 1, 3)
y_labels = torch.from_numpy(data['arr_1'])


In [4]:
len_dataset = len(y_labels) 

dataset = MyDataset(data=x_images.float(), targets=y_labels)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [int(len_dataset*0.8), len_dataset-int(len_dataset*0.8)])

train_loader = DataLoader(train_dataset, batch_size = 100, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 100, shuffle = False)


In [5]:
def validate(model, valloader, device):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            labels=labels.squeeze(1)
            # print(predicted)
            # print(labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            # print(correct / total)

    return correct / total 

In [9]:
def train(model, num_epochs=50):
    
    #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #print('Using device:', device)
    
    device = torch.device("cpu")
    
    sum_acc = np.zeros((1,  num_epochs))
    sum_loss = sum_acc.copy()
    model.train()
    model.to(device)

    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4) #change learning rate
    best_accuracy = 0
    best_loss = 100
    #fig = plt.figure() 

    for epoch in tqdm(range(num_epochs)):
        epoch_start = time.time()

        for img_batch, labels_batch in train_loader:
            optimizer.zero_grad()
            print(img_batch.shape)
            print(labels_batch.shape)
        
            output = model(img_batch.to(device))
            loss = criterion(output, labels_batch.to(device).squeeze().long())
            loss.backward()
            optimizer.step()
            images = img_batch.cpu()
            label_nums = output.cpu() 
            accuracy = validate(model, val_loader, device)
#             print(accuracy)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            print('Best accuracy improved')
            torch.save(model.state_dict(), 'model_weights.pth') #name of saved weights
        if best_loss > loss.cpu().item():
            best_loss = loss.cpu().item()
#             print('Best loss improved')

        sum_acc[0, epoch] = accuracy
        sum_loss[0,epoch] = loss  
        epoch_end = time.time()
        print("Epoch: {} Loss: {:.3f} Accuracy: {:.3f} Time: {:.4f}s".format(epoch, loss.item(),accuracy, epoch_end-epoch_start))
        
        plt.plot(sum_acc[0,0:epoch].T)
        plt.show()
    
    return sum_acc, sum_loss

In [7]:
import segmentation_models_pytorch as smp

In [13]:
#import segmentation_models_pytorch as smp

#model = smp.Unet()

#model = smp.Unet(
#    encoder_name="resnet34",        # choose encoder
#    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
#    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#    classes=3,                      # model output channels (number of classes in your dataset)
#)

model = smp.Unet(encoder_name='resnet34', 
                 encoder_depth=5, 
                 encoder_weights='imagenet', 
                 decoder_use_batchnorm=True, 
                 decoder_channels=(256, 128, 64, 32, 16), 
                 decoder_attention_type=None, 
                 in_channels=3, 
                 classes=3, 
                 activation=None, 
                 aux_params=None)

accuracy, loss = train(model, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

torch.Size([100, 256, 256, 3])
torch.Size([100, 256, 256, 3])


RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[100, 256, 256, 3] to have 3 channels, but got 256 channels instead

In [ ]:
plt.plot(accuracy.T, 'r')

In [ ]:
plt.plot(loss.T, 'b')